####  Provide your CDISc Library API key

In [ ]:
#API_KEY='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [2]:
import os
import requests
import json
import getpass

baseURL = "https://library.cdisc.org/api"

def query(queryEndpoint = "", queryPath = ""):
    resp = requests.get(
        baseURL + queryEndpoint + queryPath, 
        headers={
            'api-key': API_KEY, 
            'Accept': 'application/json'}
    )
    #print(resp.status_code)
    if resp.status_code == 200:
        return resp.json()       


In [3]:
#resp = query("/mdr/bc/packages")
resp = query("/mdr/bc/packages/2022-10-26/biomedicalconcepts")
resp_sdtm = query("/mdr/specializations/sdtm/packages/2022-10-26/datasetspecializations")

In [4]:
#for getting the detailed sdtm specialisations from API - need the name of the BC specialisation from the list
bcs_sdtm = []
for bc in resp_sdtm['_links']['datasetSpecializations']:
    name= bc.get("title") 
    ref = bc.get('href')
    ref_f=int(ref.rindex('/'))+1
    ref_t=int(len(ref))
    bc_sdtm_name=ref[ref_f:ref_t ]
    map = dict({
        'name':name,
        'term':bc_sdtm_name
        })
    bcs_sdtm.append(map)

In [5]:
#get all SDTM specialisations with details
resp_sdtm = []
for bc in bcs_sdtm:
    term = bc['term']
    responses = query('/mdr/specializations/sdtm/packages/2022-10-26/datasetspecializations/'+term)
    resp_sdtm.append(responses)
print('Done')

Done


In [6]:
#get code of all bc concepts from list of BC concepts - to be used in UI box
bcs = []
for bc in resp['_links']['biomedicalConcepts']:
    name= bc.get("title") 
    ref = bc.get('href')
    ref_f=int(ref.rindex('/'))+1
    ref_t=int(len(ref))
    bc_code=ref[ref_f:ref_t]
    map = dict({
        'name':name,
        'code':bc_code
        })
    bcs.append(map)
#print(bcs)

In [7]:
#UI for selecting BCs to display in graph view
import sys
import tkinter as tk
from tkinter import *
from tkinter import messagebox
import yaml


def resetAll():
    app.destroy()
    app = Tk()

def get_code_from_href(href):
    ref=href
    ref_f=int(ref.rindex('/'))+1
    ref_t=int(len(ref))
    code=ref[ref_f:ref_t]
    return code

def save_selected():
    global bc_lst_var
    bc_lst_var = [box.get(idx) for idx in box.curselection()]
    
    global bc_sdtm_lst
    bc_sdtm_lst = []
    if (var1.get() == 1):
        l.config(text='Adding SDTM specialisation')
        for bc in bc_lst_var:
            b = yaml.safe_load(bc)
            bc_concept_code = b['code']
            for bc_sdtm in resp_sdtm:
                bc_parent_code = get_code_from_href(bc_sdtm['_links']['parentBiomedicalConcept']['href'])
                if bc_concept_code == bc_parent_code:
                    bc_sdtm_lst.append(bc_sdtm)                  
    else:
        l.config(text='Only show concepts')
        
    print('Number of BC saved: '+str(len(bc_lst_var)))
    print('Selection of BCs: ')
    for bc in bc_lst_var:
        print(bc)
    
    if (var1.get() == 1):
        if (len(bc_sdtm_lst)>0):
            print('Number of BC SDTM specialisations saved: '+str(len(bc_sdtm_lst)))
            for bc_sdtm in bc_sdtm_lst:
                bc_parent_code = get_code_from_href(bc_sdtm['_links']['parentBiomedicalConcept']['href'])
                print({'name':bc_sdtm['shortName'],'code':bc_sdtm['datasetSpecializationId'], 'parent code': bc_parent_code})
        else:
            print('No BC SDTM specialisation exists')
    
    messagebox.showinfo('Save', 'The selected BCs have been saved for graphical display. Number of BC saved: '+str(len(bc_lst_var)))
    
    #Doesn't work on mac
    #app.destroy()
    
    
app = tk.Tk()
var1 = tk.IntVar()

l = tk.Label(app, bg='white', width=20, text='Only show concepts')
l.pack()

app.title('Select BC to inlcude in visualisation')

box = tk.Listbox(app, selectmode=tk.MULTIPLE, height=30,width= 100)

values = bcs

for val in values:
    box.insert(tk.END, val)
box.pack()

button = tk.Button(app, text='Save selection', width=25, command=save_selected)
button.pack()

button = tk.Checkbutton(app, text='Add SDTM specialisation',variable=var1, onvalue=1, offvalue=0)
button.pack()

button = tk.Button(app, text='Clear Selection',
                         command=lambda: box.selection_clear(0, 'end'))
button.pack()

m = tk.Label(app, bg='white', width=50, text='To close the window click on the x on the window')
m.pack()

#button = tk.Button(app, text='Close', width=25, command=app.destroy)
#button.pack()

app.mainloop()


Number of BC saved: 2
Selection of BCs: 
{'name': 'Glucose Measurement', 'code': 'C105585'}
{'name': 'Chemistry Test', 'code': 'C49237'}
Number of BC SDTM specialisations saved: 4
{'name': 'Glucose Concentration in Blood', 'code': 'GLUCBLD', 'parent code': 'C105585'}
{'name': 'Glucose Concentration in Plasma', 'code': 'GLUCPL', 'parent code': 'C105585'}
{'name': 'Glucose Concentration in Serum', 'code': 'GLUCSER', 'parent code': 'C105585'}
{'name': 'Glucose Concentration in Urine', 'code': 'GLUCURIN', 'parent code': 'C105585'}


In [20]:
#get the bc concepts selected
import yaml
resp = []
for bc in bc_lst_var:
    #To get bc's as a dictionary not a string - so we can use b['code'] to get the code.
    b = yaml.safe_load(bc)
    code = b['code']
    responses = query("/mdr/bc/packages/2022-10-26/biomedicalconcepts/"+code)
    resp.append(responses)
print('Done')

Done


In [21]:
from yfiles_jupyter_graphs import GraphWidget
w = GraphWidget()

In [50]:
#Making nodes and edges from the BCs (and BC sdtm specialisation if chosen in the UI)
from typing import Union, List, Dict
from itertools import chain

def custom_node_styles_mapping(index: int, node: Dict):
    return {'shape':'pill',}

def make_y_nodes(o:Union[List, Dict]) -> List:
    if isinstance(o, dict):
        o = [dict]
    return [{
        "id": x.get("href"), 
        "properties": {**x, **{'label': (x.get("shortName") if x.get("shortName") else x.get("title"))}}
    } for x in o]

def make_y_sdtm_nodes(o:Union[List, Dict]) -> List:
    if isinstance(o, dict):
        o = [dict]
    return [{
        "id": x.get('_links').get('self').get("href"), 
        "properties": {**{k: item for k, item in x.items() if not k in ["_links"]},**x.get('_links').get('self'), **{'label': (x.get("shortName") if x.get("shortName") else x.get("title"))}}
    } for x in o]


def make_y_nodes_items(o:Union[List, Dict]) -> List:
    if isinstance(o, dict):
        o = [dict]
    return [{
        "id": x.get("href")+'?'+self_.get("shortName"), 
        "properties": {**x, **{'label': (x.get("shortName") if x.get("shortName") else x.get("title"))}}
    } for x in o]


def make_y_nodes_sdtm_vars(o:Union[List, Dict]) -> List:
    if isinstance(o, dict):
        o = [dict]
    return [{
        "id": x.get("name")+'?'+self_.get("shortName"), 
        "properties": {**x, **{'label': (x.get("name") if x.get("name") else x.get("title"))}}
    } for x in o]

def make_y_nodes_terms(o:Union[List, Dict]) -> List:
    if isinstance(o, dict):
        o = [dict]
    return [{
        "id": x.get("term")+v.get("name")+'?'+self_.get("shortName"), 
        "properties": {**x, **{'label': x.get("term")}}
    } for x in o]


n_list=[]
e_list=[]
bc_ref_list=[]

#get the concept BC's and their items (children)
for r in resp:
    #print(r.get("_links").get('self').get('title'))
    self_ = {k: item for k, item in r.items() if not k in ["_links", "dataElementConcepts"]}
    n = make_y_nodes([self_])
    n_list.append(n) 
  
    #Find the parent of the BC if, any. Note the parent ID. To be used in the loop below for parent nodes.
    if r.get("dataElementConcepts") is not None:
        children = r.get("dataElementConcepts")
 
        n = make_y_nodes_items(children) 
        
        n_list.append(n)
        
        e = ([{'id': i, 'start': self_.get("href"), 'end': x.get("href")+'?'+self_.get("shortName"), 'properties': {'label': 'HAS_ITEM'}}
              for i, x in enumerate(children)])
        e_list.append(e)
        if r.get("_links").get('parentBiomedicalConcept') is not None:
            p_ref=r.get("_links").get('parentBiomedicalConcept').get('href')
            ref_f=int(p_ref.rindex('/'))+1
            ref_t=int(len(p_ref))
            p_code=p_ref[ref_f:ref_t]
            bc_p_href ='https://ncithesaurus.nci.nih.gov/ncitbrowser/ConceptReport.jsp?dictionary=NCI_Thesaurus&ns=ncit&code='+p_code
            bc_href =r.get('href')
            
            #Making parent-child edges          
            e=([{'id': -1, 'start': bc_p_href, 'end': bc_href, 'properties': {'label': 'HAS_CHILD'}}])
            e_list.append(e)

            
if len(bc_sdtm_lst)>0:
    for bc_sdtm in bc_sdtm_lst:
        #print(r.get("_links").get('self').get('title'))
        self_ = {k: item for k, item in bc_sdtm.items() if not k in ["variables"]}

        n = make_y_sdtm_nodes([self_])

        n_list.append(n)

    #Find the parent of the BC if, any. Note the parent ID. To be used in the loop below for parent nodes.
        if bc_sdtm.get("variables") is not None:
            children = bc_sdtm.get("variables")
            
            n = make_y_nodes_sdtm_vars(children)

            n_list.append(n)

            e = ([{'id': i, 'start': self_.get('_links').get('self').get("href"), 'end': x.get("name")+'?'+self_.get("shortName"), 'properties': {'label': 'HAS_VARIABLE'}}
                  for i, x in enumerate(children)])
            e_list.append(e)
        
        # make nodes for terms
            for v in bc_sdtm.get("variables"):
                t=[]
                if v.get('assignedTerm') is not None:
                    if v.get('codelist') is not None:
                        terms = dict({'term':v.get('assignedTerm').get('value'),'codelist':v.get('codelist').get('href')})
                        t.append(terms)
                if v.get('valueList') is not None:
                    if v.get('codelist') is not None:
                        for i in v.get('valueList'):
                            terms = dict({'term':i,'codelist':v.get('codelist').get('href')})
                            t.append(terms)
                        
                n = make_y_nodes_terms(t)
                
                n_list.append(n)
                                
        #make edges for terms to their variables
                e = ([{'id': i, 'start': v.get("name")+'?'+self_.get("shortName"), 'end': x.get("term")+v.get("name")+'?'+self_.get("shortName"), 'properties': {'label': 'HAS_TERM'}} 
                      for i, x in enumerate(t)])
            
                e_list.append(e)

            

        if bc_sdtm.get("_links").get('parentBiomedicalConcept') is not None:
            p_ref=bc_sdtm.get("_links").get('parentBiomedicalConcept').get('href')
            ref_f=int(p_ref.rindex('/'))+1
            ref_t=int(len(p_ref))
            p_code=p_ref[ref_f:ref_t]
            bc_p_href ='https://ncithesaurus.nci.nih.gov/ncitbrowser/ConceptReport.jsp?dictionary=NCI_Thesaurus&ns=ncit&code='+p_code
            bc_href =bc_sdtm.get('_links').get('self').get("href")

            #Making parent-child edges          
            e=([{'id': -1, 'start': bc_p_href, 'end': bc_href, 'properties': {'label': 'HAS_SDTM_SPECIALISATION'}}])


            e_list.append(e)



nodes = [n for node in n_list for n in node]
nodes_uniq = list(
   {
       dictionary['id']: dictionary
       for dictionary in nodes
   }.values()
)

edges=[e for edge in e_list for e in edge]
edges_uniq = list(
    {
        dictionary['id']: dictionary
        for dictionary in edges
    }.values()
)



w.nodes = nodes
w.edges=edges
w.directed = True

#nodes and edges formatting

w.node_styles_mapping=custom_node_styles_mapping
w.set_node_label_mapping(lambda index, node : node["properties"]["label"])
w.set_node_color_mapping(lambda index, node : 'grey' if 'term' in node['properties'] else 'lightblue' if (('dataElementConceptId' in node['properties'])| ('assignedTerm' in node['properties'])| ('datasetSpecializationId' in node['properties'])) else '#F5B200')
w.set_edge_color_mapping(lambda index, node  : '#122140')
w.set_node_scale_factor_mapping(lambda index, node : 1.5)
w.set_graph_layout({'algorithm': 'circular'})



In [51]:
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
#resets the formatting
w.del_node_styles_mapping()

In [ ]:
w.del_edge_color_mapping()

In [ ]:
w.del_node_label_mapping()

In [ ]:
w.del_node_color_mapping()